In [1]:
import numpy as np
import pandas as pd
import glob
import re

### South Africa Schools Data

In [2]:
def load_data(path):
    dfs = []
    fnames = glob.glob(path)
    for fname in fnames:
        df = pd.read_excel(fname)
        dfs.append(df)
    return dfs

### Columns of Interest

| Column Name | Description | Values |
| --- | --- | --- |
NatEmis | School unique Id|
|Institution_Name|School Name| |
| Status | Indicates whether school  operational or not. | {OPERATIONAL', 'PENDING CLOSURE', 'PENDING OPEN', 'PENDING '} |
| Sector | Whether private or public school | PUBLIC,INDEPENDENT |
|Type_PED | Whether ordinary school, techmolohy centre, prison, ... |{'ORDINARY SCHOOL' ,'MUSIC CENTRE' ,'EDUCATION TECHNOLOGY CENTRE',,'PRE-PRIMARY SCHOOL' ,'PRISON SCHOOL' ,'SPECIAL NEEDS EDUCATION CENTRE'}|
|Phase_PED| Is it a primary school, middle school, ... | {'99' ,'COMBINED SCHOOL' ,'HOSPITAL SCHOOL' ,'INTERMEDIATE SCHOOL','PRE-PRIMARY SCHOOL' ,'PRIMARY SCHOOL' ,'SCHOOL OF SKILLS',,'SECONDARY SCHOOL', 'SPECIAL NEEDS EDUCATION',,'SPECIAL NEEDS EDUCATION SCHOOL' ,'SPECIALIZED SCHOOL'}
|Specialization|School focus area| {ADULT BASIC EDUCATION AND TRAINING' 'AGRICULTURE' 'ARTS AND CULTURE''ARTS, DRAMA, MUSIC & BALLET' 'AUTISM' 'Art & Drama' 'Art School''Art and Museum Schools' 'Arts, drama,...}|
|ExamNo| Examination number | |
|Province| School in which province | EC,NW,MP,.....|
|DistrictMunicipalityName| District municipality name| |
|Local MunicipalityName| The local municiplaity | |
|PostalAddress|

### Column Grouping
The information can be grouped according to whether its about the school or its geographical location. Also observe the columns listed below may be processed to extract only certain information i.e postal code from postal address. Also note column mapping where applicable.
- ### School Information
 - NatEmis:Id
 - Status
 - Sector
 - Type_PED : Type
 - Phase_PED : Phase
 - Specialization
 - ExamNo:Examination_No
 
- School Location Informatiion
 - Province
 - DistrictMunicipalityName : District_Municipality
 - Local MuniciplalityName : Local_Municipality
 - PostalAddress : Postal Address

In [3]:
info_cols_raw = ['NatEmis','Institution_Name','Status','Sector','Type_PED','Phase_PED','Specialization','ExamNo']
info_cols_map = {'NatEmis':'Id','Institution_Name':'Name','Status':'Status','Sector':'Sector','Type_PED':'Type',
                           'Phase_PED':'Phase','Specialization':'Specialization',
                           'ExamNo':'ExamNo'}
info_cols = list(info_cols_map.values())
loc_cols_raw = ['Province','DistrictMunicipalityName','Local MunicipalityName','Suburb',
                              'PostalAddress']
loc_cols_map = {'Province':'Province','DistrictMunicipalityName' : 'District_Municipality',
                      'Local MunicipalityName':'Local_Municipality','Suburb':'Suburb',
                      'PostalAddress':'Postal_Code'}
loc_cols = list(loc_cols_map.values())

### Filter To Interest Columns, Rename & Merge

In [4]:
def filter_merge(dfs):
    columns = info_cols_raw + loc_cols_raw
    new_dfs = []
    for df in dfs:
        new_dfs.append(df[columns])
    
    df =  pd.concat(new_dfs,ignore_index=True).copy()
    c = {**info_cols_map,**loc_cols_map}
    df.rename(columns = c,inplace=True)
    df = df.drop_duplicates(subset='Id', keep='first')
    return df

### Cleaning 
- Info Colummns
 - Remove special characters and trim spaces then reduce to pascal case/title

In [5]:
def clean(df,cols):
    def f(x):
        y = re.sub(r"[^a-zA-Z0-9]+", ' ', str(x)).title()
        return y.strip()
    def g(x):
        data = re.split(' |,',str(x))
        if len(data)>0:
            if data[-1].strip().isnumeric():
                return data[-1].strip()
            return ''
        return ''
    
    df_c = df.copy().fillna('')
    for c in cols:
        if c!='Postal_Code':
            df_c[c] = df_c[c].apply(f)
        else:
             df_c[c] = df_c[c].apply(g)
    return df_c

### E2E

In [6]:
def process_files(path):
    dfs = load_data(path)
    df = filter_merge(dfs)
    df_c = clean(df,info_cols + loc_cols)
    return df,df_c

In [7]:
df,df_c = process_files('data/input/*.xlsx')

In [8]:
df_c.to_csv("data/output/schools.csv",index=False)

### Ready To Make A Database